In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
df = pd.read_csv("coba_data_baru.csv")

In [3]:
# dic = {"tag":[], "patterns":[], "responses":[]}
# for i in range(len(df)):
#     ptrns = df[df.index == i]['patterns'].values[0]
#     rspns = df[df.index == i]['responses'].values[0]
#     tag = df[df.index == i]['tag'].values[0]
#     for j in range(len(ptrns)):
#         dic['tag'].append(tag)
#         dic['patterns'].append(ptrns[j])
#         dic['responses'].append(rspns)

# df = pd.DataFrame.from_dict(dic)
# pd.set_option("Display.max.rows",None)
df

,tag,patterns,responses
0,greeting,Hi,['Hello there. Tell me how are you feeling tod...
1,greeting,Hey,['Hello there. Tell me how are you feeling tod...
2,greeting,Is anyone there?,['Hello there. Tell me how are you feeling tod...
3,greeting,Hi there,['Hello there. Tell me how are you feeling tod...
4,greeting,Hello,['Hello there. Tell me how are you feeling tod...
...,...,...,...
227,fact-29,How do I know if I'm unwell?,"['If your beliefs , thoughts , feelings or beh..."
228,fact-30,How can I maintain social connections? What if...,"[""A lot of people are alone right now, but we ..."
229,fact-31,What's the difference between anxiety and stress?,"[""Stress and anxiety are often used interchang..."
230,fact-32,What's the difference between sadness and depr...,"[""Sadness is a normal reaction to a loss, disa..."


In [4]:
df = pd.read_csv("data_chat_indo.csv")

In [4]:
df['tag'].unique()

array(['greeting', 'morning', 'afternoon', 'evening', 'night', 'goodbye',
       'thanks', 'no-response', 'neutral-response', 'about', 'skill',
       'creation', 'name', 'help', 'sad', 'stressed', 'worthless',
       'depressed', 'happy', 'casual', 'anxious', 'not-talking', 'sleep',
       'scared', 'death', 'understand', 'done', 'suicide', 'hate-you',
       'hate-me', 'default', 'jokes', 'repeat', 'wrong', 'stupid',
       'location', 'something-else', 'friends', 'ask', 'problem',
       'no-approach', 'learn-more', 'user-agree', 'meditation',
       'user-meditation', 'pandora-useful', 'user-advice',
       'learn-mental-health', 'mental-health-fact', 'fact-1', 'fact-2',
       'fact-3', 'fact-5', 'fact-6', 'fact-7', 'fact-8', 'fact-9',
       'fact-10', 'fact-11', 'fact-12', 'fact-13', 'fact-14', 'fact-15',
       'fact-16', 'fact-17', 'fact-18', 'fact-19', 'fact-20', 'fact-21',
       'fact-22', 'fact-23', 'fact-24', 'fact-25', 'fact-26', 'fact-27',
       'fact-28', 'fact-29', '

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(lower=True, split=' ')
tokenizer.fit_on_texts(df['patterns'])
tokenizer.get_config()

AttributeError: 'float' object has no attribute 'lower'

In [26]:
vacab_size = len(tokenizer.word_index)
print('number of unique words = ', vacab_size)

number of unique words =  303


In [27]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

ptrn2seq = tokenizer.texts_to_sequences(df['patterns'])
X = pad_sequences(ptrn2seq, padding='post')
print('X shape = ', X.shape)

lbl_enc = LabelEncoder()
y = lbl_enc.fit_transform(df['tag'])
print('y shape = ', y.shape)
print('num of classes = ', len(np.unique(y)))

X shape =  (232, 18)
y shape =  (232,)
num of classes =  80


In [16]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, LSTM, LayerNormalization, Dense, Dropout
from tensorflow.keras.utils import plot_model

model = Sequential()
model.add(Input(shape=(X.shape[1])))
model.add(Embedding(input_dim=vacab_size+1, output_dim=100, mask_zero=True))
model.add(LSTM(32, return_sequences=True))
model.add(LayerNormalization())
model.add(LSTM(32, return_sequences=True))
model.add(LayerNormalization())
model.add(LSTM(32))
model.add(LayerNormalization())
model.add(Dense(128, activation="relu"))
model.add(LayerNormalization())
model.add(Dropout(0.2))
model.add(Dense(128, activation="relu"))
model.add(LayerNormalization())
model.add(Dropout(0.2))
model.add(Dense(len(np.unique(y)), activation="softmax"))
model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])

model.summary()
plot_model(model, show_shapes=True)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 18, 100)           27700     
                                                                 
 lstm_3 (LSTM)               (None, 18, 32)            17024     
                                                                 
 layer_normalization_5 (Laye  (None, 18, 32)           64        
 rNormalization)                                                 
                                                                 
 lstm_4 (LSTM)               (None, 18, 32)            8320      
                                                                 
 layer_normalization_6 (Laye  (None, 18, 32)           64        
 rNormalization)                                                 
                                                                 
 lstm_5 (LSTM)               (None, 32)               

In [29]:
model_history = model.fit(x=X,
                          y=y,
                          batch_size=10,
                          callbacks=[tensorflow.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)],
                          epochs=50)

Epoch 1/50
 1/24 [>.............................] - ETA: 0s - loss: 5.9414 - accuracy: 0.0000e+00

InvalidArgumentError: Graph execution error:

Detected at node 'sequential_1/embedding_1/embedding_lookup' defined at (most recent call last):
    File "C:\Users\ASUS\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\ASUS\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\ASUS\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\ASUS\anaconda3\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\ASUS\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 736, in start
      self.io_loop.start()
    File "C:\Users\ASUS\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\ASUS\anaconda3\lib\asyncio\base_events.py", line 601, in run_forever
      self._r

In [20]:
import re
import random

def inputan_user(pattern):
    # Handle empty or non-alphabetic input
    if not pattern or not any(c.isalpha() for c in pattern):
        print("User: {}".format(pattern))
        print(" BOT : Please provide a valid input.")
        return

    text = []
    txt = re.sub('[^a-zA-Z\']', ' ', pattern)
    txt = txt.lower()
    txt = txt.split()
    txt = " ".join(txt)
    text.append(txt)

    x_test = tokenizer.texts_to_sequences(text)
    x_test = np.array(x_test).squeeze()
    x_test = pad_sequences([x_test], padding='post', maxlen=X.shape[1])
    y_pred = model.predict(x_test)
    y_pred = y_pred.argmax()
    tag = lbl_enc.inverse_transform([y_pred])[0]

    # Check if the tag exists in the DataFrame
    if tag in df['tag'].values:
        responses = df[df['tag'] == tag]['responses'].values[0]
        print("User: {}".format(pattern))
        print(" BOT : {}".format(random.choice(responses)))
    else:
        print("User: {}".format(pattern))
        print(" BOT : I'm not sure how to respond to that.")


In [21]:
inputan_user('hai')

ValueError: `sequences` must be a list of iterables. Found non-iterable: 44

In [12]:
inputan_user('Iam happy today')

1/1 [==============================] - 0s 34ms/step
User: Iam happy today
 BOT : Did something happen which made you feel this way?


In [13]:
inputan_user('may i ask a question?')

1/1 [==============================] - 0s 35ms/step
User: may i ask a question?
 BOT : I understand that it can be scary. Tell me more about it.


In [14]:
inputan_user("what is depression?")

1/1 [==============================] - 0s 33ms/step
User: what is depression?
 BOT : A mental health disorder characterised by persistently depressed mood or loss of interest in activities, causing significant impairment in daily life.


In [15]:
inputan_user("How do i know if i have Depression?	")

1/1 [==============================] - 0s 34ms/step
User: How do i know if i have Depression?	
 BOT : If your beliefs , thoughts , feelings or behaviours have a significant impact on your ability to function in what might be considered a normal or ordinary way, it would be important to seek help.


In [16]:
inputan_user("Are there cures for mental health problems?	")

1/1 [==============================] - 0s 37ms/step
User: Are there cures for mental health problems?	
 BOT : It is often more realistic and helpful to find out what helps with the issues you face. Talking, counselling, medication, friendships, exercise, good sleep and nutrition, and meaningful occupation can all help.


In [17]:
inputan_user("How can I maintain social connections?")

1/1 [==============================] - 0s 37ms/step
User: How can I maintain social connections?
 BOT : Yes, sure. How can I help you?


In [18]:
inputan_user("I'm interested in learning about mental health")

1/1 [==============================] - 0s 33ms/step
User: I'm interested in learning about mental health
 BOT : Oh that's really great. I'd be willing to answer anything that I know about it.


In [20]:
# # Menyimpan model setelah pelatihan
# model.save('my_model.keras')